# Dyson V2 Deposit & Premium Simulation
此範例模擬 Dyson V2 的存款 (Deposit) 與對應的 Premium 計算流程，
並展示如何使用 Python 進行簡單的狀態維護與測試。

In [692]:
from datetime import datetime, timezone
from typing import Optional, Union
import time

class TimeManager:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(TimeManager, cls).__new__(cls)
            cls._instance._mock_time = None
            cls._instance._tz = timezone.utc
        return cls._instance

    def setCurrentTime(
        self, timestamp: Union[int, float, datetime, None] = None
    ) -> None:
        """Set current time for testing/mocking"""
        if isinstance(timestamp, datetime):
            self._mock_time = timestamp.timestamp()
        else:
            self._mock_time = timestamp

    def getCurrentTime(self) -> datetime:
        """Get current time (mocked or real)"""
        if self._mock_time is not None:
            return datetime.fromtimestamp(self._mock_time, tz=self._tz)
        return datetime.now(tz=self._tz)

    def getTimestamp(self) -> float:
        """Get Unix timestamp"""
        return self.getCurrentTime().timestamp()

    def setTimezone(self, tz: timezone) -> None:
        """Set timezone for time operations"""
        self._tz = tz

    def resetMock(self) -> None:
        """Reset mock time to use real time"""
        self._mock_time = None


# Usage example:
tm = TimeManager()
tm.setCurrentTime(datetime(2025, 4, 1, 16, 00, tzinfo=timezone.utc))
print(tm.getTimestamp())  # 2024-01-01 12:00:00+00:00

1743523200.0


In [693]:
import math
import numpy as np

########################################
# 初始參數設定
########################################

# 池子初始狀態
x = 100.0       # ETH
y = 200000.0    # USDC

# x = 1000.0  # ETH
# y = 1000.0  # USDC

# 目標當日可賣額度，設為 200 表示當日可賣相當於 200 倍底池的數量
daily_sell_target = 200.0

# 協定計算需要的基礎變數
k = math.sqrt(x * y)  # k = 底池流動性刻度
w = daily_sell_target * k  # w = 當日可賣流動性刻度

# 全域變數 sqrt_Q (此週期累積已賣的流動性刻度)
accumulatd_sqrt_Q = 0.0  

basis = 0.5  # 假設年化波動率 50%
ln2 = 0.69314718055994530941723212145817656807550013436025525412068000949339362196 # ln(2)

### DysonV2 Premium 計算
- q_old = deposit 前，當前週期內已累積的 sqrt_Q 值（已賣額度）
- q_new = deposit 後，當前週期內已累積的 sqrt_Q 值（已賣額度）
- w = 池子可賣額度

$$
a = \frac{q_{old}}{w}, b = \frac{q_{new}}{w}
$$
$$
discount = ln(2) \times \frac{ \log_2(1+b) - \log_2(1+a) }{ b - a}
$$

$$
Premium
= \underbrace{0.4\,basis\,\sqrt{t}}_{\text{basic rate}}
\times
\underbrace{\bigl(\text{discount}\bigr)}_{\text{區段平均調整}}.
$$

In [694]:
import math


def calc_discount(a, b):
    if b <= a:
        raise ValueError("b should greater than a.")

    if a < 0 or b < 0:
        # 不預期情況: 此函式假設 a, b >= 0
        raise ValueError("a, b should be non-negative in this context.")

    # 如果 a > 0, 走正常公式:
    numerator = (math.log2(b + 1) - math.log2(a + 1)) * math.log(2)
    denom = b - a
    return numerator / denom

# --- 測試 ---
test_cases = [
    (0, 0.001),
    (0, 0.1),
    (0, 0.2),
    (0.2, 0.3),
    (0.2, 0.9),
    (0.9, 0.999),
    (0.9, 1.0),
]

for a, b in test_cases:
    d_log2 = calc_discount(a, b)
    print(f"[{a}, {b}] => discount={d_log2:.4f}")

[0, 0.001] => discount=0.9995
[0, 0.1] => discount=0.9531
[0, 0.2] => discount=0.9116
[0.2, 0.3] => discount=0.8004
[0.2, 0.9] => discount=0.6565
[0.9, 0.999] => discount=0.5131
[0.9, 1.0] => discount=0.5129


In [695]:
def calc_premium(q_old, q_new, basis, duration_in_sec):
    """
    最終 Premium = (0.4 * basis * sqrt(time)) * discount
    discount = (對應區間 [q_old, q_new] 的對數平均)
    """
    discount = calc_discount(q_old/w, q_new/w)

    t_yr = duration_in_sec / (365.0 * 86400)
    basic_rate = 0.4 * basis * math.sqrt(t_yr)

    return basic_rate * discount

# --- 測試 ---
test_cases = [
    (0, 0.0005, 86400),
    (0.0005, 0.001, 86400),
]

for sqrt_Q_old, sqrt_Q_new, time in test_cases:
    premium = calc_premium(a, b, basis, time)
    print(f"[{sqrt_Q_old}, {sqrt_Q_new}, {time}] => premium={premium:.4f}")

[0, 0.0005, 86400] => premium=0.0105
[0.0005, 0.001, 86400] => premium=0.0105


In [696]:
def compute_due_time_and_duration(lock_days: int):
    """
    根據當前時間與鎖倉天數，計算到期的 UTC 日期（以天為單位）與實際鎖倉時間（秒數）
    """

    current_timestamp = tm.getTimestamp()

    seconds_in_day = 86400
    today_midnight = (current_timestamp // seconds_in_day) * seconds_in_day

    lock_start = (
        today_midnight
        if current_timestamp == today_midnight
        else today_midnight + seconds_in_day
    )
    unlock_time = lock_start + lock_days * seconds_in_day

    due_in_utc_day = unlock_time // seconds_in_day
    lock_duration_in_sec = unlock_time - current_timestamp

    return due_in_utc_day, lock_duration_in_sec

# --- 測試 ---
tm = TimeManager()
# 模擬在非午夜整點建倉，鎖倉時間設定 1 天，但實際鎖倉期間為 1 天又 8 小時：
tm.setCurrentTime(datetime(2025, 4, 1, 16, 0, tzinfo=timezone.utc))
print(compute_due_time_and_duration(1))  

# 模擬在午夜整點時建倉，鎖倉時間設定 1 天，實際鎖倉期間也為 1 天：
tm.setCurrentTime(datetime(2025, 4, 1, 0, 0, tzinfo=timezone.utc))
print(compute_due_time_and_duration(1))

(20181.0, 115200.0)
(20180.0, 86400.0)


### DysonV2 deposit 公式
$$
Q = 4 \cdot (\sqrt{(x+input_0)(y+input_1)} - \sqrt{xy})^2
$$

In [697]:
def dyson_deposit_formula(balance0, balance1, inputAmt0, inputAmt1, lockDays):
    global accumulatd_sqrt_Q
    global x, y

    """
    依照 Dyson V2 公式計算:
    Q = 4 * ( sqrt((x+input0)*(y+input1)) - sqrt(x*y) )^2
    並回傳 (noteAmt0, noteAmt1, Q)
    """
    sqrt_xy = math.sqrt(balance0 * balance1)
    sqrt_xMyN = math.sqrt((balance0 + inputAmt0) * (balance1 + inputAmt1))
    diff = max(sqrt_xMyN - sqrt_xy, 0)
    Q = 4.0 * diff * diff

    # 判斷單邊比較: inputAmt0 / balance0 vs. inputAmt1 / balance1
    # 等價於 inputAmt0*balance1 > inputAmt1*balance0
    if (inputAmt0 * balance1) > (inputAmt1 * balance0):
        ratio = (inputAmt1 * balance0) / balance1 if balance1 > 0 else 0
        noteAmt0 = inputAmt0 + ratio
        if noteAmt0 == 0:
            noteAmt1 = 0
        else:
            noteAmt1 = Q / noteAmt0
    else:
        ratio = (inputAmt0 * balance1) / balance0 if balance0 > 0 else 0
        noteAmt1 = inputAmt1 + ratio
        if noteAmt1 == 0:
            noteAmt0 = 0
        else:
            noteAmt0 = Q / noteAmt1

    sqrt_Q = math.sqrt(Q)

    # 協定更新 q：
    q_old = accumulatd_sqrt_Q
    q_new = accumulatd_sqrt_Q + sqrt_Q  # 新的刻度

    # 計算該筆投資的 Premium
    (due_in_utc_day, lock_duration_in_sec) = compute_due_time_and_duration(lockDays)
    premium = calc_premium(
        q_old=q_old, q_new=q_new, basis=basis, duration_in_sec=lock_duration_in_sec
    )

    # 最後更新全域狀態
    accumulatd_sqrt_Q = q_new
    # 同時池子 x, y 也應更新 (模擬實際存入後的新餘額)
    x += inputAmt0
    y += inputAmt1

    return (
        noteAmt0,
        noteAmt1,
        sqrt_Q,
        q_old,
        q_new,
        premium,
        due_in_utc_day,
        lock_duration_in_sec,
    )

In [698]:
def compute_withdraw_ratio(x, y, m, n):
    """
    計算使用者兌換比例 a，範圍應落在 [0, 1]。
    其中：
        x, y 為池子目前的 token0 和 token1 儲備
        m, n 為票據 note 中的 token0Amt, token1Amt
    """
    if m == 0 or n == 0:
        raise ValueError("m and n must be non-zero")

    a = 0.5 + x / (2 * m) - y / (2 * n)
    return max(0, min(a, 1))  # Clamp to [0, 1]

### Demo: 實際跑一筆 deposit

In [699]:
import pandas as pd

pd.set_option("display.float_format", lambda x: "%.12f" % x)

# Set current time for testing
tm = TimeManager()
tm.setCurrentTime(datetime(2025, 4, 1, 16, 00, tzinfo=timezone.utc))

# Create initial parameters DataFrame
initial_params = pd.DataFrame(
    {
        "Parameter": [
            "Pool ETH (x)",
            "Pool USDC (y)",
            "Daily Sell Target (ETH)",
            "K (sqrt(x*y))",
            "W (K * Daily Sell Target)",
            "Current q",
            "Basis (Annual Volatility)",
        ],
        "Value": [x, y, daily_sell_target_eth, k, w, accumulatd_sqrt_Q, basis],
    }
)

In [700]:
def process_deposit(deposit_time_days, inputAmt0, inputAmt1):
    (
        noteAmt0,
        noteAmt1,
        sqrt_Q,
        q_old,
        q_new,
        premium,
        due_in_utc_day,
        lock_duration_in_sec,
    ) = dyson_deposit_formula(x, y, inputAmt0, inputAmt1, deposit_time_days)

    noteAmt0WithPremium = noteAmt0 * (1 + premium)
    noteAmt1WithPremium = noteAmt1 * (1 + premium)

    a = compute_withdraw_ratio(x, y, noteAmt0WithPremium, noteAmt1WithPremium)

    withdraw0 = noteAmt0WithPremium * a
    withdraw1 = noteAmt1WithPremium * (1 - a)

    return {
        "Days": deposit_time_days,
        "Input(ETH, USDC)": (inputAmt0, round(inputAmt1, 4)),
        "sqrt_Q": sqrt_Q,
        "sqrt_Q_old/w, sqrt_Q_new/w": (q_old / w, q_new / w),
        "total_Q": accumulatd_sqrt_Q,
        "Note(ETH, USDC)": (noteAmt0, round(noteAmt1, 4)),
        "DueInUTC": datetime.fromtimestamp(
            due_in_utc_day * 86400, tz=timezone.utc
        ).strftime("%Y-%m-%d %H:%M:%S"),
        "DueInUTC Day": due_in_utc_day,
        "Lock Duration (sec)": lock_duration_in_sec,
        "Lock Duration (days)": lock_duration_in_sec / 86400,
        "Premium": premium,
        "NoteWithPremiun(ETH, USDC)": (
            noteAmt0WithPremium,
            noteAmt1WithPremium,
        ),
        "withdraw(ETH, USDC)": (withdraw0, round(withdraw1, 4)),
        "New Pool(ETH, USDC)": (x, y),
    }

In [701]:
print("\n=== Initial Parameters ===")
initial_params


=== Initial Parameters ===


,Parameter,Value
0,Pool ETH (x),100.000000000000
1,Pool USDC (y),200000.000000000000
2,Daily Sell Target (ETH),200.000000000000
3,K (sqrt(x*y)),4472.135954999580
4,W (K * Daily Sell Target),894427.190999915940
5,Current q,0.000000000000
6,Basis (Annual Volatility),0.500000000000


In [702]:
# Initialize results DataFrame
deposits_df = pd.DataFrame()

# First deposit
deposit1 = process_deposit(1, 10, 0)
deposits_df = pd.concat([deposits_df, pd.DataFrame([deposit1])], ignore_index=True)

# Second deposit
deposit2 = process_deposit(1, 10, 0)
deposits_df = pd.concat([deposits_df, pd.DataFrame([deposit2])], ignore_index=True)

print("=== Deposit Results ===")
deposits_df

=== Deposit Results ===


,Days,"Input(ETH, USDC)",sqrt_Q,"sqrt_Q_old/w, sqrt_Q_new/w",total_Q,"Note(ETH, USDC)",DueInUTC,DueInUTC Day,Lock Duration (sec),Lock Duration (days),Premium,"NoteWithPremiun(ETH, USDC)","withdraw(ETH, USDC)","New Pool(ETH, USDC)"
0,1,"(10, 0)",436.559609647700,"(0.0, 0.0004880884817015153)",436.559609647700,"(10.0, 19058.4293)",2025-04-03 00:00:00,20181.000000000000,115200.000000000000,1.333333333333,0.012085008667,"(10.120850086674047, 19288.75056058185)","(7.5902038390832, 4823.0143)","(110.0, 200000.0)"
1,1,"(10, 0)",417.127451485852,"(0.0004880884817015153, 0.000954451150103321)",853.687061133553,"(10.0, 17399.5311)",2025-04-03 00:00:00,20181.000000000000,115200.000000000000,1.333333333333,0.012079245528,"(10.12079245527885, 17609.704286273165)","(7.587582994949034, 4407.6657)","(120.0, 200000.0)"


# Withdraw after certain condition

### (user1 deposit) -> (user2 deposit) -> (user2 withdraw) -> (user1 withdraw)
If user2 withdraw first, how much can user1 withdraw?

In [703]:
(noteAmt0WithPremium, noteAmt1WithPremium) = deposits_df.iloc[0][
    "NoteWithPremiun(ETH, USDC)"
]
(user2Withdraw0, user2Withdraw1) = deposits_df.iloc[1]["withdraw(ETH, USDC)"]
a = compute_withdraw_ratio(x-user2Withdraw0, y-user2Withdraw1, noteAmt0WithPremium, noteAmt1WithPremium)

user1Withdraw0 = noteAmt0WithPremium * a
user1Withdraw1 = noteAmt1WithPremium * (1 - a)

print(f"After User2 withdraw, User1 can withdraw ETH: {user1Withdraw0}, USDC: {user1Withdraw1}")

After User2 withdraw, User1 can withdraw ETH: 9.952768312975708, USDC: 320.3374596879189


### (user deposit) -> (pool rebalance) -> (user withdraw)
If pool has been rebalanced, how much can user1 withdraw?

In [704]:

def rebalance(ethBalance, usdcBalance):
    ethValue = ethBalance * 2000
    totalValue = ethValue + usdcBalance
    targetVallue = totalValue / 2

    ethAmountToSwap = 0
    usdcAmountToSwap = 0

    if ethValue > targetVallue:
        excessValue = ethValue - targetVallue
        ethAmountToSwap = excessValue / 2000
        usdcAmountToReceive = excessValue
        return ethBalance - ethAmountToSwap, usdcBalance + usdcAmountToReceive
    elif usdcBalance > targetVallue:
        excessValue = usdcBalance - targetVallue
        ethAmountToReceive = excessValue / 2000
        usdcAmountToSwap = excessValue
        return ethBalance + ethAmountToReceive, usdcBalance - usdcAmountToSwap

(newEthBalance, newUsdcBalance) = rebalance(110, 200000)
print(f"New ETH Balance: {newEthBalance}, New USDC Balance: {newUsdcBalance}")

New ETH Balance: 105.0, New USDC Balance: 210000.0


In [705]:
(ethBalance, usdcBalance) = deposits_df.iloc[0]["New Pool(ETH, USDC)"]
(newEthBalance, newUsdcBalance) = rebalance(ethBalance, usdcBalance)
print(f"New ETH Balance: {newEthBalance}, New USDC Balance: {newUsdcBalance}")

(noteAmt0WithPremium, noteAmt1WithPremium) = deposits_df.iloc[0][
    "NoteWithPremiun(ETH, USDC)"
]
a = compute_withdraw_ratio(
    newEthBalance, newUsdcBalance, noteAmt0WithPremium, noteAmt1WithPremium
)

user1Withdraw0 = noteAmt0WithPremium * a
user1Withdraw1 = noteAmt1WithPremium * (1 - a)
print(f"User1 can withdraw ETH: {user1Withdraw0}, USDC: {user1Withdraw1}")

New ETH Balance: 105.0, New USDC Balance: 210000.0
User1 can withdraw ETH: 2.4666927788705104, USDC: 14587.621573020935


In [706]:
def to_q64_64(value: float) -> str:
    """
    將十進位浮點數轉換成 Q64.64 fixed point 十六進位表示。
    Q64.64 意味著乘以 2^64，再轉成十六進位。
    """
    q64_val = int(value * (1 << 64))  # value * 2^64
    return hex(q64_val)


# 範例：
print(
    "ln(2):",
    to_q64_64(
        0.69314718055994530941723212145817656807550013436025525412068000949339362196
    ),
)  # ➜ 0xb17217f7d1cf79ac
print("pi:   ", to_q64_64(3.141592653589793))  # ➜ 0x3243f6a8885a308d
print("e:    ", to_q64_64(2.718281828459045))  # ➜ 0x2b7e151628aed2a6

ln(2): 0xb17217f7d1cf7800
pi:    0x3243f6a8885a30000
e:     0x2b7e151628aed2000
